In [58]:
from lxml import etree
import json
import time
import os
import cv2
import math

import pandas as pd
import numpy as np

In [70]:
json_path = 'D:\\datasets\\nut\\trainval.json'
image_path = 'D:\\datasets\\nut\\images\\'
target_path = 'D:\\datasets\\nut\\odtk\\'

In [74]:
with open(json_path,'r') as f:
    json_dict = json.load(f)

In [75]:
#calc_bearing是一个用arctan求θ的简单函数。你必须把函数封装起来以确保w和h是正的，并且theta在-pi/2到pi/2或者-pi到pi的范围内。
def calc_bearing(pointA, pointB):
    delta_x = -abs(pointA[0]-pointB[0])
    delta_y = -abs(pointA[1]-pointB[1])
#     if delta_x<delta_y:
#         bearing = math.atan2(delta_x,delta_y)
#         print(1,delta_x,delta_y)
#     else:
#         bearing = math.atan2(delta_y,delta_x)
#         print(2,delta_x,delta_y)
    bearing = math.atan2(delta_y,delta_x)
    return bearing

def _corners2rotatedbbox(corners):
    centre = np.mean(np.array(corners), 0)
    theta = calc_bearing(corners[0], corners[1])
    rotation = np.array([[np.cos(theta), -np.sin(theta)],
                        [np.sin(theta), np.cos(theta)]])
    out_points = np.matmul(corners - centre, rotation) + centre
    x, y = list(out_points[0,:])
    w, h = list(out_points[2, :] - out_points[0, :])
    return [x, y, w, h, theta]

In [95]:
anno_list = json_dict['annotations']

for i in anno_list:
    seg = i['segmentation']
    cnt = np.reshape(np.float32(seg),(-1,2))
    rect = cv2.minAreaRect(cnt)
    box = cv2.boxPoints(rect)
    
    rbox = _corners2rotatedbbox(box)
    i['bbox'] = list(rbox)
    i['segmentation'] = list(box.flatten())

In [96]:
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(NpEncoder, self).default(obj)

In [97]:
with open('D:\\datasets\\nut\\rbox_trainval.json','w') as f:
    json.dump(json_dict,f, cls=NpEncoder)